In [ ]:
# word2vec 설명 문서
# https://radimrehurek.com/gensim/models/word2vec.html

# 5.12(화) 수업 참조 site - 챗봇을 위한 seq2seq
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
# http://www.manythings.org/anki/
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [34]:
import numpy as np
import pandas as pd

In [10]:
df = pd.read_csv("C:/Users/student/김건우/Python 데이터 분석/데이터 분석 3/수업/data/fra.txt", sep="\t", header=None)
df

del df[2]
df.columns = ['eng', 'fra']
df

,eng,fra
0,Go.,Va !
1,Hi.,Salut !
2,Hi.,Salut.
3,Run!,Cours !
4,Run!,Courez !
...,...,...
175618,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
175619,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
175620,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
175621,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


In [11]:
df = df[0:70000]
df.head(10)

,eng,fra
0,Go.,Va !
1,Hi.,Salut !
2,Hi.,Salut.
3,Run!,Cours !
4,Run!,Courez !
5,Who?,Qui ?
6,Wow!,Ça alors !
7,Fire!,Au feu !
8,Help!,À l'aide !
9,Jump.,Saute.


In [12]:
# fra 열 전체에 대해 시그널 추가 작업 수행
# start : "\t ", stop: " \n"
# ex) Rattrape-le. -> \t Rattrape-le. \n

df['fra'] = df['fra'].apply(lambda x: "\t " + x + " \n")
df

C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,eng,fra
0,Go.,\t Va ! \n
1,Hi.,\t Salut ! \n
2,Hi.,\t Salut. \n
3,Run!,\t Cours ! \n
4,Run!,\t Courez ! \n
...,...,...
69995,I've never ridden a horse.,\t Je ne suis jamais monté à cheval. \n
69996,I've never ridden a horse.,\t Je ne suis jamais monté sur un cheval. \n
69997,I've never seen Tom drunk.,\t Je n'ai jamais vu Tom ivre. \n
69998,I've never seen a rainbow.,\t Je n'ai jamais vu d'arc-en-ciel. \n


In [14]:
# 글자 단위 토큰화
engVocab = set()
fraVocab = set()

for line in df.eng:
    for c in line:
        engVocab.add(c)
        
for line in df.fra:
    for c in line:
        fraVocab.add(c)

In [16]:
engVocabSize = len(engVocab) + 1 # 인덱스 1부터 시작하려고 +1
print(engVocabSize)

fraVocabSize = len(fraVocab) + 1
print(fraVocabSize)

80
106


In [18]:
engVocab = sorted(list(engVocab))
engVocab[50:80]

fraVocab = sorted(list(fraVocab))
fraVocab[50:80]

['Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«']

In [19]:
engToIndex = dict([(c,i+1) for i,c in enumerate(engVocab)])
engToIndex

fraToIndex = dict([(c,i+1) for i,c in enumerate(fraVocab)])
fraToIndex

{'\t': 1,
 '\n': 2,
 ' ': 3,
 '!': 4,
 '"': 5,
 '$': 6,
 '%': 7,
 '&': 8,
 "'": 9,
 '(': 10,
 ')': 11,
 ',': 12,
 '-': 13,
 '.': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 '?': 26,
 'A': 27,
 'B': 28,
 'C': 29,
 'D': 30,
 'E': 31,
 'F': 32,
 'G': 33,
 'H': 34,
 'I': 35,
 'J': 36,
 'K': 37,
 'L': 38,
 'M': 39,
 'N': 40,
 'O': 41,
 'P': 42,
 'Q': 43,
 'R': 44,
 'S': 45,
 'T': 46,
 'U': 47,
 'V': 48,
 'W': 49,
 'X': 50,
 'Y': 51,
 'Z': 52,
 'a': 53,
 'b': 54,
 'c': 55,
 'd': 56,
 'e': 57,
 'f': 58,
 'g': 59,
 'h': 60,
 'i': 61,
 'j': 62,
 'k': 63,
 'l': 64,
 'm': 65,
 'n': 66,
 'o': 67,
 'p': 68,
 'q': 69,
 'r': 70,
 's': 71,
 't': 72,
 'u': 73,
 'v': 74,
 'w': 75,
 'x': 76,
 'y': 77,
 'z': 78,
 '\xa0': 79,
 '«': 80,
 '»': 81,
 'À': 82,
 'Ç': 83,
 'É': 84,
 'Ê': 85,
 'Ô': 86,
 'à': 87,
 'â': 88,
 'ç': 89,
 'è': 90,
 'é': 91,
 'ê': 92,
 'ë': 93,
 'î': 94,
 'ï': 95,
 'ô': 96,
 'ù': 97,
 'û': 98,
 'œ': 99,
 'С': 100,
 '\

In [20]:
# 영어 문장 -> 정수 인코딩
encoderInput=[]
for li in df.eng:
    
    t=[]
    for c in li:
        
        t.append(engToIndex[c])
        
    encoderInput.append(t)
    
print(encoderInput[:10])

[[30, 64, 10], [31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2], [46, 57, 64, 23], [46, 64, 72, 2], [29, 58, 67, 54, 2], [31, 54, 61, 65, 2], [33, 70, 62, 65, 10]]


In [21]:
# 프랑스어 문장 -> 정수 인코딩
decoderInput=[]
for li in df.fra:
    
    t=[]
    for c in li:
        t.append(fraToIndex[c])
        
    decoderInput.append(t)
    
print(decoderInput[:10])

[[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2], [1, 3, 29, 67, 73, 70, 71, 105, 4, 3, 2], [1, 3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2], [1, 3, 43, 73, 61, 3, 26, 3, 2], [1, 3, 83, 53, 3, 53, 64, 67, 70, 71, 105, 4, 3, 2], [1, 3, 27, 73, 3, 58, 57, 73, 3, 4, 3, 2], [1, 3, 82, 3, 64, 9, 53, 61, 56, 57, 105, 4, 3, 2], [1, 3, 45, 53, 73, 72, 57, 14, 3, 2]]


In [22]:
# 프랑스어 문장 출력
decoderFra=[]
for li in df.fra:
    
    t=[]
    i=0
    for c in li:
        
        if i > 0:
            t.append(fraToIndex[c])
            
        i+=1
        
    decoderFra.append(t)
    
print(decoderFra[:10])

[[3, 48, 53, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2], [3, 29, 67, 73, 70, 71, 105, 4, 3, 2], [3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2], [3, 43, 73, 61, 3, 26, 3, 2], [3, 83, 53, 3, 53, 64, 67, 70, 71, 105, 4, 3, 2], [3, 27, 73, 3, 58, 57, 73, 3, 4, 3, 2], [3, 82, 3, 64, 9, 53, 61, 56, 57, 105, 4, 3, 2], [3, 45, 53, 73, 72, 57, 14, 3, 2]]


In [23]:
decoderInput

[[1, 3, 48, 53, 3, 4, 3, 2],
 [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2],
 [1, 3, 45, 53, 64, 73, 72, 14, 3, 2],
 [1, 3, 29, 67, 73, 70, 71, 105, 4, 3, 2],
 [1, 3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2],
 [1, 3, 43, 73, 61, 3, 26, 3, 2],
 [1, 3, 83, 53, 3, 53, 64, 67, 70, 71, 105, 4, 3, 2],
 [1, 3, 27, 73, 3, 58, 57, 73, 3, 4, 3, 2],
 [1, 3, 82, 3, 64, 9, 53, 61, 56, 57, 105, 4, 3, 2],
 [1, 3, 45, 53, 73, 72, 57, 14, 3, 2],
 [1, 3, 83, 53, 3, 71, 73, 58, 58, 61, 72, 105, 4, 3, 2],
 [1, 3, 45, 72, 67, 68, 105, 4, 3, 2],
 [1, 3, 27, 70, 70, 92, 72, 57, 13, 72, 67, 61, 3, 4, 3, 2],
 [1, 3, 27, 72, 72, 57, 66, 56, 71, 3, 4, 3, 2],
 [1, 3, 27, 72, 72, 57, 66, 56, 57, 78, 3, 4, 3, 2],
 [1, 3, 42, 67, 73, 70, 71, 73, 61, 71, 14, 3, 2],
 [1, 3, 29, 67, 66, 72, 61, 66, 73, 57, 78, 14, 3, 2],
 [1, 3, 42, 67, 73, 70, 71, 73, 61, 74, 57, 78, 14, 3, 2],
 [1, 3, 28, 67, 66, 62, 67, 73, 70, 3, 4, 3, 2],
 [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2],
 [1, 3, 36, 57, 3, 55, 67, 65, 68, 70, 57, 66, 56, 71, 14, 3, 

In [25]:
# 문장 -> 정수 변환 -> 패딩
# 패딩 : 각 언어별로 가장 긴 글자로 구성된 문장 길이로 통일
maxEngLen = max([len(li) for li in df.eng]) # 영어문장의 가장 긴 문자의 길이
print(maxEngLen)

maxFraLen = max([len(li) for li in df.fra]) 
print(maxFraLen)

26
76


In [31]:
# 인코더와 디코더의 문자의 길이가 같을 필요는 없음
from keras.preprocessing.sequence import pad_sequences

encoderInput = pad_sequences(encoderInput, maxlen=maxEngLen, padding='post') # post : 뒷쪽
decoderInput = pad_sequences(decoderInput, maxlen=maxFraLen, padding='post')
decoderFra = pad_sequences(decoderFra, maxlen=maxFraLen, padding='post')

In [36]:
np.shape(encoderInput)
np.shape(decoderInput)

(70000, 76)

In [37]:
# 원핫인코딩
from keras.utils import to_categorical

encoderInput = to_categorical(encoderInput)
decoderInput = to_categorical(decoderInput)
decoderFra = to_categorical(decoderFra)

In [42]:
np.shape(encoderInput) # (70000, 26, 80) (전체 문장수, 전체 줄에서 최대 글자수, 글자의 종류)
np.shape(decoderInput) # (70000, 76, 106) 

(70000, 76, 106)

In [43]:
# 트레이닝시 이전 상태의 실제값을 현재 상태의 디코더 입력으로 해야함(예측값으로 하면 안됨)
from keras.layers import Input, Embedding, Dense, LSTM
from keras.models import Model

In [ ]:
# ex) ip = Input(shape=(50,1)) -> feature : 1개, time-step(시점) : 50개
# it = LSTM(출력)(ip)
# d1 = Dense(10, activation='relu')(it)
# d2 = Dense(1, activation='sigmoid')(d1)
# Model(inputs=ip, outputs=d2)

In [49]:
encoderInputs = Input(shape=(None, engVocabSize))
decoderInputs = Input(shape=(None, fraVocabSize)) # 디코더 입력 = Input(None(maxFraLen), 프랑스어 문자 종류 수)

# 인코더 LSTM 셀
encoderLSTM = LSTM(units=256, return_state=True) 
# 256 : 은닉상태 크기
# return_state=True : 인코더의 마지막 상태 정보를 디코더의 입력 상태 정보로 전달 옵션

# 디코더 LSTM 셀
decoderLSTM = LSTM(units=256, return_sequences=True, return_state=True)

# 인코더 LSTM셀의 입력 정의
_, stateH, stateC = encoderLSTM(encoderInputs) # _(인코더의 출력은 필요없음), 히든상태(위), 셀상태(오른쪽)
encoderStates = [stateH, stateC] # 컨텍스트 벡터

decoderOutputs, _, _ = decoderLSTM(decoderInputs, initial_state=encoderStates)
decoderSoftmax = Dense(fraVocabSize, activation="softmax")
decoderOutputs = decoderSoftmax(decoderOutputs)

In [50]:
model = Model(inputs=[encoderInputs,decoderInputs], outputs=decoderOutputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, None, 80)     0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, None, 106)    0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 256), (None, 345088      input_11[0][0]                   
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, None, 256),  371712      input_12[0][0]                   
                                                                 lstm_5[0][1]               

In [52]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit(x=[encoderInput,decoderInput], y=decoderFra, batch_size=64, epochs=50, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 56000 samples, validate on 14000 samples
Epoch 1/50
56000/56000 [==============================] - 347s 6ms/step - loss: 0.7591 - val_loss: 0.6816
Epoch 2/50
56000/56000 [==============================] - 322s 6ms/step - loss: 0.4724 - val_loss: 0.5482
Epoch 3/50
56000/56000 [==============================] - 348s 6ms/step - loss: 0.3910 - val_loss: 0.4848
Epoch 4/50
56000/56000 [==============================] - 310s 6ms/step - loss: 0.3478 - val_loss: 0.4503
Epoch 5/50
56000/56000 [==============================] - 310s 6ms/step - loss: 0.3197 - val_loss: 0.4245
Epoch 6/50
56000/56000 [==============================] - 310s 6ms/step - loss: 0.2994 - val_loss: 0.4076
Epoch 7/50
56000/56000 [==============================] - 307s 5ms/step - loss: 0.2841 - val_loss: 0.3959
Epoch 8/50
56000/56000 [==============================] - 309s 6ms/step - loss: 0.2717 - val_loss: 0.3875
Epoch 9

In [53]:
# 모델 저장
from keras.models import load_model

model.save('ENG_FRA_model.h5')

In [54]:
model = load_model('ENG_FRA_model.h5')

In [ ]:
"""
<seq2seq 동작>

1) 입력 문장 -> 인코더 -> 은닉상태, 셀 상태 얻어냄
2) 상태정보와 Start 시그널("\t")을 디코더로 전달
3) 다음 문자를 예측(stop 시그널("\n")이 등장할 때 까지 반복)
"""
# Train : 디코더 입력으로 정답을 넣어줌 / Test : 이전 셀의 출력을 입력으로 받음 

In [167]:
encoderModel = Model(inputs=encoderInputs, outputs=encoderStates)

In [168]:
# 디코더
decoderStateInputHidden = Input(shape=(256,))
decoderStateInputCell = Input(shape=(256,))
decoderStateInputs = [decoderStateInputHidden, decoderStateInputCell]

decoderOutputs, stateHidden, stateCell = decoderLSTM(decoderInputs, initial_state=decoderStateInputs)
decoderStates = [stateHidden, stateCell]
decoderOutputs = decoderSoftmax(decoderOutputs)

decoderModel = Model(inputs=[decoderInputs]+decoderStateInputs, outputs=[decoderOutputs]+decoderStates)

In [169]:
indexToEng = dict((i,c) for c,i in engToIndex.items())
indexToFra = dict((i,c) for c,i in fraToIndex.items())

In [170]:
def decodeSeq(inputSeq): # (1, 26, 80)
    
    statesValue = encoderModel.predict(inputSeq)
    # print(statesValue)
    # print(np.shape(statesValue))
    
    targetSeq = np.zeros((1,1,fraVocabSize)) # 1,1,106
    targetSeq[0,0,fraToIndex['\t']] = 1 # 원핫인코딩
    
    stop = False
    decodedSent=""
    while not stop: # "\n"문자를 만날때까지 반복
        
        output, h, c = decoderModel.predict([targetSeq]+statesValue)
        # 예측값을 프랑스 문자로 변환
        tokenIndex = np.argmax(output[0,-1,:]) 
        predChar = indexToFra[tokenIndex]
        
        # 현시점 예측문자가 예측문장에 추가
        decodedSent+=predChar
        
        if (predChar=="\n" or len(decodedSent)>maxFraLen):
            stop = True
            
        # 현시점 예측결과가 다음 시점에 입력으로 
        targetSeq = np.zeros((1,1,fraVocabSize))
        targetSeq[0,0,tokenIndex] = 1
        
        # 현시점 상태를 다음 시점 상태로 사용
        statesValue = [h,c]
    
    return decodedSent # 번역결과

In [173]:
for seqIndex in [1,50,100,200,300]:
    
    inputSeq = encoderInput[seqIndex:seqIndex+1]
    # print(np.shape(inputSeq)) # (1, 26, 80)
    decodedSeq = decodeSeq(inputSeq)
    
    print("입력문장:", df.eng[seqIndex])
    print("정답:", df.fra[seqIndex][1:len(df.fra[seqIndex])-1]) # "\t", "\n" 제거
    print("번역기:", decodedSeq[:len(decodedSeq)-1])
    print("\n")

입력문장: Hi.
정답:  Salut ! 
번역기:  Salut. 


입력문장: I lied.
정답:  J'ai menti. 
번역기:  J'ai perdu. 


입력문장: Come in.
정답:  Entre. 
번역기:  Entrez ! 


입력문장: Skip it.
정답:  Pas grave. 
번역기:  Prends les ordures. 


입력문장: I did OK.
정답:  Je m'en suis bien sortie. 
번역기:  Je me suis bien occupé. 




array([[[0., 0., 0., 0., 0.]]])

In [111]:
# 텍스트 제너레이션
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

text = """과수원에 있는 배가 맛있다
그의 배는 많이 나왔다
가는 길에 배를 탔고 오는 길에도 배를 탔다"""

In [112]:
t = Tokenizer()
t.fit_on_texts([text])

In [113]:
vocSize = len(t.word_index) + 1
# 1~15까지 존재, 원핫인코딩하면 인덱스가 0부터 -> 배열의 크기를 단어집합의 크기보다 +1

In [114]:
seqs=[]
for line in text.split("\n"):
    encoded = t.texts_to_sequences([line])[0]
    
    for i in range(1,len(encoded)):
        seq = encoded[:i+1]
        seqs.append(seq)

seqs

[[2, 3],
 [2, 3, 4],
 [2, 3, 4, 5],
 [6, 7],
 [6, 7, 8],
 [6, 7, 8, 9],
 [10, 11],
 [10, 11, 1],
 [10, 11, 1, 12],
 [10, 11, 1, 12, 13],
 [10, 11, 1, 12, 13, 14],
 [10, 11, 1, 12, 13, 14, 1],
 [10, 11, 1, 12, 13, 14, 1, 15]]

In [115]:
maxLen = max(len(i) for i in seqs)

seqs = pad_sequences(seqs, maxlen=maxLen, padding='pre')
seqs

array([[ 0,  0,  0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  0,  0,  2,  3,  4],
       [ 0,  0,  0,  0,  2,  3,  4,  5],
       [ 0,  0,  0,  0,  0,  0,  6,  7],
       [ 0,  0,  0,  0,  0,  6,  7,  8],
       [ 0,  0,  0,  0,  6,  7,  8,  9],
       [ 0,  0,  0,  0,  0,  0, 10, 11],
       [ 0,  0,  0,  0,  0, 10, 11,  1],
       [ 0,  0,  0,  0, 10, 11,  1, 12],
       [ 0,  0,  0, 10, 11,  1, 12, 13],
       [ 0,  0, 10, 11,  1, 12, 13, 14],
       [ 0, 10, 11,  1, 12, 13, 14,  1],
       [10, 11,  1, 12, 13, 14,  1, 15]])

In [116]:
seqs = np.array(seqs)

x = seqs[:,:-1]
y = seqs[:,-1]

y = to_categorical(y, num_classes=vocSize)

In [119]:
from keras.layers import SimpleRNN
from keras.models import Sequential

# 모델 구성
model = Sequential()
model.add(Embedding(vocSize, 10, input_length=maxLen-1)) # (입력차원, 출력차원, 입력길이) / 레이블 분리해서 -1
model.add(SimpleRNN(32)) # 각 단어의 임베딩 벡터가 10차원, 32개 히든 상태
model.add(Dense(vocSize, activation='softmax'))

In [123]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.fit(x, y, epochs=200)

Epoch 1/200
13/13 [==============================] - 0s 22ms/step - loss: 2.7718 - accuracy: 0.1538
Epoch 2/200
13/13 [==============================] - 0s 1ms/step - loss: 2.7600 - accuracy: 0.1538
Epoch 3/200
13/13 [==============================] - 0s 0us/step - loss: 2.7499 - accuracy: 0.1538
Epoch 4/200
13/13 [==============================] - 0s 307us/step - loss: 2.7380 - accuracy: 0.1538
Epoch 5/200
13/13 [==============================] - 0s 308us/step - loss: 2.7259 - accuracy: 0.1538
Epoch 6/200
13/13 [==============================] - 0s 308us/step - loss: 2.7135 - accuracy: 0.1538
Epoch 7/200
13/13 [==============================] - 0s 308us/step - loss: 2.7013 - accuracy: 0.1538
Epoch 8/200
13/13 [==============================] - 0s 0us/step - loss: 2.6884 - accuracy: 0.1538
Epoch 9/200
13/13 [==============================] - 0s 308us/step - loss: 2.6751 - accuracy: 0.1538
Epoch 10/200
13/13 [==============================] - 0s 308us/step - loss: 2.6615 - accuracy: 0.2

Epoch 83/200
13/13 [==============================] - 0s 308us/step - loss: 1.2901 - accuracy: 0.5385
Epoch 84/200
13/13 [==============================] - 0s 0us/step - loss: 1.2739 - accuracy: 0.5385
Epoch 85/200
13/13 [==============================] - 0s 307us/step - loss: 1.2579 - accuracy: 0.5385
Epoch 86/200
13/13 [==============================] - 0s 308us/step - loss: 1.2422 - accuracy: 0.6154
Epoch 87/200
13/13 [==============================] - 0s 0us/step - loss: 1.2268 - accuracy: 0.6154
Epoch 88/200
13/13 [==============================] - 0s 472us/step - loss: 1.2117 - accuracy: 0.6154
Epoch 89/200
13/13 [==============================] - 0s 0us/step - loss: 1.1968 - accuracy: 0.6154
Epoch 90/200
13/13 [==============================] - 0s 308us/step - loss: 1.1821 - accuracy: 0.6923
Epoch 91/200
13/13 [==============================] - 0s 308us/step - loss: 1.1678 - accuracy: 0.6923
Epoch 92/200
13/13 [==============================] - 0s 0us/step - loss: 1.1537 - accur

Epoch 164/200
13/13 [==============================] - 0s 308us/step - loss: 0.4217 - accuracy: 1.0000
Epoch 165/200
13/13 [==============================] - 0s 0us/step - loss: 0.4148 - accuracy: 1.0000
Epoch 166/200
13/13 [==============================] - 0s 308us/step - loss: 0.4080 - accuracy: 1.0000
Epoch 167/200
13/13 [==============================] - 0s 308us/step - loss: 0.4013 - accuracy: 1.0000
Epoch 168/200
13/13 [==============================] - 0s 307us/step - loss: 0.3947 - accuracy: 1.0000
Epoch 169/200
13/13 [==============================] - 0s 0us/step - loss: 0.3883 - accuracy: 1.0000
Epoch 170/200
13/13 [==============================] - 0s 308us/step - loss: 0.3820 - accuracy: 1.0000
Epoch 171/200
13/13 [==============================] - 0s 308us/step - loss: 0.3757 - accuracy: 1.0000
Epoch 172/200
13/13 [==============================] - 0s 308us/step - loss: 0.3696 - accuracy: 1.0000
Epoch 173/200
13/13 [==============================] - 0s 308us/step - loss: 

In [133]:
def sentGen(model, t, word, n): # 모델, 토큰나이저, 입력단어, 예측단어 갯수
    
    sent=""
    word2 = word
    
    for _ in range(n): # n번 반복
        
        encoded = t.texts_to_sequences([word])[0]
        encoded = pad_sequences([encoded], maxlen=7, padding='pre')
        res = model.predict_classes(encoded)
        
        for w, i in t.word_index.items():
            
            if i == res: # 예측단어와 인덱스가 동일한 단어
                break
                
        word = word + " " + w # "과수원에 있는"
        sent = sent + " " + w
            
    sent = word2 + sent
    
    return sent

In [142]:
sentGen(model, t, '과수원에', 3) # 과수원에 뒤에 등장하는 3개의 단어 예측

'과수원에 있는 배가 맛있다'